In [1]:
import os
import json
from difflib import unified_diff
from pyspark import ml
import numpy as np
cwd = os.getcwd()
print(cwd)

C:\Users\u0115374\Documents\PhD\Courses\Big Data\SparkStream


In [2]:
def make_diff(old, new):
    return '\n'.join([ l for l in unified_diff(old.split('\n'), new.split('\n')) if l.startswith('+') or l.startswith('-') ])

In [3]:
'''Function to extract the added pieces of text in the document.'''
def get_added_text(diff):
    edits = []
    for text in diff.split('\n'):
        if text.startswith("+"):
            edits.append(text)
    #print(edits)        
    return edits

In [4]:
from pyspark import SparkContext
sc = SparkContext("local", "example")

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('example').getOrCreate()
spark

In [6]:
#df = spark.read.format("json").load(cwd+"\Assignment3\\streaming_1a\\outputs-*\\part-*")


os.chdir('C:\\Users\\u0115374\\Documents\\PhD\\Courses\\Big Data\\Assignment3\\streaming_1a')
cwd = os.getcwd()

In [7]:
dirs = list(os.scandir(cwd))
dirs

[<DirEntry 'naive_bayes'>,
 <DirEntry 'outputs-1585746250000'>,
 <DirEntry 'outputs-1585746260000'>,
 <DirEntry 'outputs-1585746270000'>,
 <DirEntry 'outputs-1585746280000'>,
 <DirEntry 'outputs-1585746290000'>,
 <DirEntry 'outputs-1585746300000'>,
 <DirEntry 'outputs-1585746310000'>,
 <DirEntry 'outputs-1585746320000'>,
 <DirEntry 'outputs-1585746330000'>,
 <DirEntry 'outputs-1585746340000'>,
 <DirEntry 'outputs-1585746350000'>,
 <DirEntry 'outputs-1585746360000'>,
 <DirEntry 'outputs-1585746370000'>,
 <DirEntry 'outputs-1585746380000'>,
 <DirEntry 'outputs-1585746390000'>,
 <DirEntry 'outputs-1585746400000'>,
 <DirEntry 'outputs-1585746410000'>,
 <DirEntry 'outputs-1585746420000'>,
 <DirEntry 'outputs-1585746430000'>,
 <DirEntry 'outputs-1585746440000'>,
 <DirEntry 'outputs-1585746450000'>,
 <DirEntry 'outputs-1585746460000'>,
 <DirEntry 'outputs-1585746470000'>,
 <DirEntry 'outputs-1585746480000'>,
 <DirEntry 'outputs-1585746490000'>,
 <DirEntry 'outputs-1585746500000'>,
 <DirEntry 

In [8]:
dirs = os.listdir(cwd)

In [9]:
df = spark.read.format("json").load(cwd + '//' + dirs[4000])
df.printSchema()

root
 |-- comment: string (nullable = true)
 |-- label: string (nullable = true)
 |-- name_user: string (nullable = true)
 |-- text_new: string (nullable = true)
 |-- text_old: string (nullable = true)
 |-- title_page: string (nullable = true)
 |-- url_page: string (nullable = true)



In [10]:
'''for i in range(len(dirs)):
    df_tojoin = spark.read.format("json").load(cwd + '//' + dirs[i])
    dataframe = df.union(df_tojoin)'''

'for i in range(len(dirs)):\n    df_tojoin = spark.read.format("json").load(cwd + \'//\' + dirs[i])\n    dataframe = df.union(df_tojoin)'

In [11]:
cwd

'C:\\Users\\u0115374\\Documents\\PhD\\Courses\\Big Data\\Assignment3\\streaming_1a'

In [12]:
df = spark.read.format("json").load(cwd + '//*//*' )

In [13]:
df.printSchema()

root
 |-- comment: string (nullable = true)
 |-- label: string (nullable = true)
 |-- name_user: string (nullable = true)
 |-- text_new: string (nullable = true)
 |-- text_old: string (nullable = true)
 |-- title_page: string (nullable = true)
 |-- url_page: string (nullable = true)



In [14]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
udfMake_Diff = udf(make_diff, StringType())
df_new = df.withColumn("diff", udfMake_Diff("text_old", "text_new"))

In [15]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, RegexTokenizer
tokenizer = RegexTokenizer(inputCol="diff", outputCol="words", pattern="\\W")
tokenizer2 = RegexTokenizer(inputCol="diff", outputCol="words", pattern="\\d")
df_tknz = tokenizer.transform(df_new)

In [16]:
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
df_filter = remover.transform(df_tknz)

In [17]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="filtered", outputCol="vectors")
model = cv.fit(df_filter)
df_cv = model.transform(df_filter)
df_cv.printSchema()

root
 |-- comment: string (nullable = true)
 |-- label: string (nullable = true)
 |-- name_user: string (nullable = true)
 |-- text_new: string (nullable = true)
 |-- text_old: string (nullable = true)
 |-- title_page: string (nullable = true)
 |-- url_page: string (nullable = true)
 |-- diff: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- vectors: vector (nullable = true)



In [18]:
idf = IDF(inputCol="vectors", outputCol="features")
idfModel = idf.fit(df_cv)
df_tfidf = idfModel.transform(df_cv)

In [19]:
df_tfidf.printSchema()

root
 |-- comment: string (nullable = true)
 |-- label: string (nullable = true)
 |-- name_user: string (nullable = true)
 |-- text_new: string (nullable = true)
 |-- text_old: string (nullable = true)
 |-- title_page: string (nullable = true)
 |-- url_page: string (nullable = true)
 |-- diff: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- vectors: vector (nullable = true)
 |-- features: vector (nullable = true)



In [20]:
# Decision Tree Algorithm
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler, StringIndexer

label_indexer = StringIndexer(inputCol = "label", outputCol = "target")
data = label_indexer.fit(df_tfidf).transform(df_tfidf).select("target", "features")

In [21]:
train, test = data.randomSplit([0.70, 0.30])
train.printSchema()

root
 |-- target: double (nullable = false)
 |-- features: vector (nullable = true)



In [22]:
#dt = DecisionTreeClassifier(labelCol="target", featuresCol="features")
#modeldt = dt.fit(train)
#predictiondt = modeldt.transform(test)

In [23]:
#predictiondt.printSchema()

In [24]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=1.0, modelType="multinomial", labelCol="target", featuresCol="features")
model_nb = nb.fit(train)
predictions = model_nb.transform(test)
evaluator = MulticlassClassificationEvaluator(labelCol="target", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.6176191122573639


In [25]:
#data.write.parquet('data_1a.parquet')
train.write.parquet("train")
#test.saveAsTextFile('test.txt')

AnalysisException: 'path file:/C:/Users/u0115374/Documents/PhD/Courses/Big Data/SparkStream/train already exists.;'

In [42]:
cwd

'C:\\Users\\u0115374\\Documents\\PhD\\Courses\\Big Data\\Assignment3\\streaming_1a'

In [54]:
train.write.parquet("C:\\Users\\u0115374\\Desktop\\training_data.parquet")

Py4JJavaError: An error occurred while calling o1424.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOper

In [66]:
model_nb.save(cwd)

Py4JJavaError: An error occurred while calling o1969.save.
: java.io.IOException: Path C:\Users\u0115374\Documents\PhD\Courses\Big Data\Assignment3\streaming_1a already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:702)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:179)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(Unknown Source)
	at java.lang.reflect.Method.invoke(Unknown Source)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execut